In [1]:
import pandas as pd

price_data = pd.read_csv("../Task 1/Natural Gas Data Forecast Result.csv", parse_dates=['Dates'])

In [2]:
def contract_price(injection_dates, withdrawal_dates, injection_rate, withdrawal_rate, max_storage, yearly_storage_cost):
    injection_data = price_data[price_data['Dates'].isin(injection_dates)].copy()
    withdrawal_data = price_data[price_data['Dates'].isin(withdrawal_dates)].copy()
    injection_data['Transaction']=injection_rate
    withdrawal_data['Transaction']=-withdrawal_rate
    transaction_data = pd.concat([injection_data,withdrawal_data])
    transaction_data = transaction_data.sort_values(by='Dates')
    transaction_data = transaction_data.reset_index(drop=True)
    
    cum_vol = 0
    for index,row in transaction_data.iterrows():
        cum_vol += row['Transaction']
        if cum_vol > max_storage or cum_vol < 0: 
            cum_vol -= row['Transaction']
            transaction_data.at[index,'Transaction'] = 0
            
    transaction_data['Transaction_Prices'] = transaction_data['Transaction'] * transaction_data['Prices'] 
            
    count_days = int((transaction_data['Dates'].iloc[-1]-transaction_data['Dates'].iloc[0]).days)
    storage_cost = count_days * yearly_storage_cost/365.25
    
    contract_cost = -transaction_data['Transaction_Prices'].sum()-storage_cost
    print(contract_cost)

In [3]:
contract_price(('2020-12-31','2021-05-31'),('2020-12-31','2025-09-30'),10,10,100,0.5)

22.067939630699655
